In [ ]:
import cv2
import numpy as np
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import os

In [2]:
BEST_MODEL_PATH = r"path_to\weights\best.pt"

In [3]:
model = YOLO(BEST_MODEL_PATH)

In [4]:
def apply_segment(image_path, to_path):
    results = model.predict(image_path)
    # results[0].show()
    try:
        for result in results:
            # Get the height and width of the original image
            height, width = result.orig_img.shape[:2]

            # Create the background
            background = np.ones((height, width, 3), dtype=np.uint8) * 255

            # Get all predicted masks
            masks = result.masks.xy

            # Get the original image
            orig_img = result.orig_img

            for mask in masks:
                    mask = mask.astype(int)

                    # Create a mask image
                    mask_img = np.zeros_like(orig_img)

                    # Fill the contour of the mask image in white
                    cv2.fillPoly(mask_img, [mask], (255, 255, 255))

                    # Extract the object from the original image using the mask
                    masked_object = cv2.bitwise_and(orig_img, mask_img)

                    # Copy the masked object to the background image
                    background[mask_img == 255] = masked_object[mask_img == 255]
    except:
        print("ERROR:", image_path)
        return
    cv2.imwrite(to_path, background)

## Apply to all

In [ ]:
path = r"path_to_data"
# ALL_PATHS = [path + "\\val\\" +  folder for folder in list(os.walk(path + "\\val"))[0][1]] +  [path + "\\train\\" +  folder for folder in list(os.walk(path + "\\val"))[0][1]]
ALL_PATHS = [path + "\\" +  folder for folder in list(os.walk(path))[0][1]]
ALL_PATHS

In [ ]:
for path in ALL_PATHS:
    i = 0
    filenames = [i for i in os.listdir(path) if i.endswith(('.png', '.jpg', '.jpeg'))]
    for image in filenames:
        image_path = os.path.join(path, image)
        print(image_path)
        apply_segment(
            image_path, image_path)
        i += 1
    print(i, " done in ", path)